In [1]:
from pyspark.sql import SparkSession
from pyspark import  SparkContext
import os
import pandas as pd

/tmp/ipykernel_64879/2081191832.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
spark = SparkSession.builder.master("yarn").getOrCreate()

/spark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/21 13:34:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/21 13:35:01 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [8]:
import pandas as pd

# Sample data for Project table
project_data = {
    'project_id': [1, 1, 2, 2],
    'employee_id': [1, 2, 3, 4],
    'workload': [45, 90, 12, 68]
}

# Sample data for Employees table
employees_data = {
    'employee_id': [1, 2, 3, 4],
    'name': ['Khaled', 'Ali', 'John', 'Doe'],
    'team': ['A', 'B', 'B', 'A']
}

# Create DataFrame for Project
project_df = pd.DataFrame(project_data)

# Create DataFrame for Employees
employees_df = pd.DataFrame(employees_data)

# Display the DataFrames
print("Project table:")
print(project_df)
print("\nEmployees table:")
print(employees_df)


df_person = spark.createDataFrame(project_df)
df_person.createOrReplaceTempView("Project")

df_person = spark.createDataFrame(employees_df)
df_person.createOrReplaceTempView("Employees")



Project table:
   project_id  employee_id  workload
0           1            1        45
1           1            2        90
2           2            3        12
3           2            4        68

Employees table:
   employee_id    name team
0            1  Khaled    A
1            2     Ali    B
2            3    John    B
3            4     Doe    A


In [15]:
query = """
    select 
        *
    from (
        select
            e.employee_id,
            p.project_id,
            e.name as employee_name,
            p.workload as project_workload,
            avg(p.workload) over(partition by e.team) as average_workload
        from Employees e left join Project p on e.employee_id = p.employee_id
    ) where project_workload > average_workload
"""

In [16]:
spark.sql(query).show()

+-----------+----------+-------------+----------------+----------------+
|employee_id|project_id|employee_name|project_workload|average_workload|
+-----------+----------+-------------+----------------+----------------+
|          4|         2|          Doe|              68|            56.5|
|          2|         1|          Ali|              90|            51.0|
+-----------+----------+-------------+----------------+----------------+

